# Importing Modules and Defining Functions

In [147]:
import re
import chess
import chess.pgn
import sys
import requests
import pandas as pd
from json import loads
import matplotlib.pyplot as plt

In [142]:
def httpsReq(requestAddr):

    #Making the API Request
    # print(f'Making API Call to: {requestAddr}')
    response = requests.get(requestAddr)
    if response.status_code == 200:
        pass
    elif response.status_code != 200:
        print(f'{response.status_code} - API Request {requestAddr} Failed')
        KeyboardInterrupt

    #Converting JSON to Dict and Parsing API Data into a Dataframe
    api_dict = loads(response.text)

    return api_dict

In [2]:
#Function for Writing .xlsx Data Tables which will provide a standard output for table formatting 
def xlsx_table_writer(data_table,sheet_name,col_width_list,title_str,startrow,startcol,worksheet,workbook,writer):

    #Layout/Formatting
    t_vert_spacing = 5 #Vertical Spacing between tables
    t_horz_spacing = 1 #Horizontal Spacing between tables
    title_format = workbook.add_format({'bold': True, 'font_size' : 20, 'fg_color' : '#76933C', 'font_color' : 'white' }) #Standardised Title Format for all tables
    header_format = workbook.add_format({'bold' : True, 'font_size' : 12, 'text_wrap' : True}) #text-wrapping for table headers
    
    # Adding and changing active sheet
    try:
        worksheet=workbook.add_worksheet(sheet_name)
        writer.sheets[sheet_name] = worksheet 
        worksheet.set_zoom(70)
        startrow = 3
        startcol = 1
    except:
        pass

    for i in range(len(col_width_list)):
        worksheet.set_column(i+startcol, i+startcol, col_width_list[i])

    #Writing in Cell Data and Merging Cells for Table Titles
    data_table.to_excel(writer,sheet_name=sheet_name,startrow=startrow , startcol=startcol, index=False, header=False)
    worksheet.merge_range(startrow-2, startcol,startrow-2,data_table.shape[1] + startcol -1, title_str,title_format) #writing in title formatting above table

    #Column settings to use in add table function
    column_settings = [{'header' : column} for column in data_table.columns]

    #Populating Excel with Table Format - Adding table to xls for each df
    worksheet.add_table(startrow-1, startcol, startrow + data_table.shape[0], data_table.shape[1] + startcol - 1, {'columns' : column_settings, 'style': 'Table Style Medium 4', 'autofilter' : False})   

    #Applying a text wrap to the Column Header
    for col_num, value in enumerate(data_table.columns.values):
        worksheet.write(startrow-1, col_num + startcol, value, header_format)
    
    #Setting Positions of Following Tables Insertions
    startrow = startrow + data_table.shape[0] + t_vert_spacing #Setting to start row for next table
    # startrow = 3  
    # startcol = startcol + data_table.shape[1] + t_horz_spacing  #Disabling horizontally displaced tables in favour of vertically displacements
    startcol = 1

    #Setting the column width at the end of the table to keep to spacing minimal between tables
    worksheet.set_column(startcol-1,startcol-1,1)
    
    #Return the start row in order to index for future function calls
    return [startrow,startcol,worksheet,workbook,writer]

In [3]:
#Function to Detect Operating System and Adjust Pathing to Respective Filesystem
def pathing(folder_path,filename):

    #Windows Operating System
    if 'win' in sys.platform:
        if folder_path == 'root':
            filepath = f'{sys.path[0]}\\{filename}'
        else:
            filepath = f'{sys.path[0]}\\{folder_path}\\{filename}'
    #Linux/Mac Operating Sytem
    else:
        if folder_path == 'root':
            filepath = f'{sys.path[0]}/{filename}'
        else:
            filepath = f'{sys.path[0]}/{folder_path}/{filename}'

    return filepath

# Requesting Game History via Chess.com API
### We need to stage our api request to transform into a dataframe:
- The api request address only returns the games for the selected month and year
- There we must pull down our archive histotry and loop through the api requests to return data for each month

In [143]:
#Retrieveing Archive of My Entire Chess.com History
player = 'filiplivancic'
dict = httpsReq(f'https://api.chess.com/pub/player/{player}/games/archives')
print(f'API Request for Monthly API Endpoints Successful')
archive = (dict['archives'])
archive[:5]

API Request for Monthly API Endpoints Successful


['https://api.chess.com/pub/player/filiplivancic/games/2019/05',
 'https://api.chess.com/pub/player/filiplivancic/games/2019/07',
 'https://api.chess.com/pub/player/filiplivancic/games/2019/08',
 'https://api.chess.com/pub/player/filiplivancic/games/2019/09',
 'https://api.chess.com/pub/player/filiplivancic/games/2019/10']

In [25]:
api_list = [httpsReq(request) for request in archive]
print(f'API Request Successful and Appended all request links together')

API Request Successful and Appended all the dictionaries together


## Unpacking List of Games

In [149]:
game_list = []
print('Transforming JSON data into a dataframe')
for month in api_list: 
    month_of_games = month['games']
    for game in month_of_games: 
        game_list.append(game)
df = pd.DataFrame(game_list)
df[-3:]

Transforming JSON data into a dataframe


,url,pgn,time_control,end_time,rated,tcn,uuid,initial_setup,fen,time_class,rules,white,black,accuracies
2495,https://www.chess.com/game/live/72393817055,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180+2,1678655218,True,mC0KgvZRfA!Tlt70vM6SjrSAtA3VMv5Qcl0SiqTCbsCldl...,bc29df60-c118-11ed-8269-6cfe544c0428,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,8/4kpp1/7p/8/7P/4q1K1/4q3/8 w - -,blitz,chess,"{'rating': 788, 'result': 'checkmated', '@id':...","{'rating': 787, 'result': 'win', '@id': 'https...",NaN
2496,https://www.chess.com/game/live/72440644947,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180+2,1678702027,True,mC0SgvZJCJ7Jfm!Teg5QbsJ7jr6Zcj90fe8!lBTJsJSJkA...,0a89baab-c186-11ed-8269-6cfe544c0428,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,1n1r1r1k/p1p1R1Qp/2p5/8/1P1P1p2/P6P/RB3PP1/6K1...,blitz,chess,"{'rating': 810, 'result': 'win', '@id': 'https...","{'rating': 780, 'result': 'checkmated', '@id':...",NaN
2497,https://www.chess.com/game/live/72443086645,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180+2,1678704555,True,mCZJgvJCvM1TMC6LdNLUNvUCvC5QfH7Rbs86ltWOHyQBel...,c6fa4a83-c18b-11ed-8269-6cfe544c0428,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,R6r/1kprp2p/Q5p1/1p6/1P6/2Pq1pPP/5P2/2K4R b - -,blitz,chess,"{'rating': 788, 'result': 'win', '@id': 'https...","{'rating': 751, 'result': 'checkmated', '@id':...",NaN


#### Tidying pgn data and consolidating into dataframe

In [237]:
pgn_list = [df.loc[i,'pgn'].splitlines(False) for i in range(df.shape[0])]
for i,pgn in enumerate(pgn_list):

    for j in range(len(pgn)):
        pgn_list[i][j] = pgn_list[i][j].split(' \"')

        for k in range(len(pgn_list[i][j])):
            pgn_list[i][j][k] = pgn_list[i][j][k].replace('\"','')
            pgn_list[i][j][k] = pgn_list[i][j][k].replace('[','')
            pgn_list[i][j][k] = pgn_list[i][j][k].replace(']','')

pgn_list[1]

[['Event', 'Live Chess'],
 ['Site', 'Chess.com'],
 ['Date', '2019.05.03'],
 ['Round', '-'],
 ['White', 'Mazterja5'],
 ['Black', 'filiplivancic'],
 ['Result', '0-1'],
 ['CurrentPosition', '2k1r2r/1pp4p/3p1p1n/P2P1n2/4K1B1/6q1/1P1B4/R2Q4 w - -'],
 ['Timezone', 'UTC'],
 ['ECO', 'A45'],
 ['ECOUrl', 'https://www.chess.com/openings/Indian-Game-2.Nc3'],
 ['UTCDate', '2019.05.03'],
 ['UTCTime', '16:49:51'],
 ['WhiteElo', '797'],
 ['BlackElo', '925'],
 ['TimeControl', '600'],
 ['Termination', 'filiplivancic won by checkmate'],
 ['StartTime', '16:49:51'],
 ['EndDate', '2019.05.03'],
 ['EndTime', '17:02:55'],
 ['Link', 'https://www.chess.com/game/live/3668084540'],
 [''],
 ['1. d4 {%clk 0:09:59.7} 1... Nf6 {%clk 0:09:56.1} 2. Nc3 {%clk 0:09:55.7} 2... Nc6 {%clk 0:09:51.2} 3. d5 {%clk 0:09:48.5} 3... Ne5 {%clk 0:09:47.7} 4. f4 {%clk 0:09:44.2} 4... Neg4 {%clk 0:09:37.1} 5. h3 {%clk 0:09:36.3} 5... Nh6 {%clk 0:09:00.8} 6. e4 {%clk 0:09:27.5} 6... e6 {%clk 0:08:18.1} 7. e5 {%clk 0:09:02.9} 7... Nfg8

#### Retrieving pgn headers

In [197]:
for pgn in pgn_list[0]:
    headers = [pgn[0] if len(pgn[0]) < 20 else 'Chess Moves' for pgn in pgn_list[0]]
print(headers)

['Event', 'Site', 'Date', 'Round', 'White', 'Black', 'Result', 'CurrentPosition', 'Timezone', 'ECO', 'ECOUrl', 'UTCDate', 'UTCTime', 'WhiteElo', 'BlackElo', 'TimeControl', 'Termination', 'StartTime', 'EndDate', 'EndTime', 'Link', '', 'Chess Moves']


#### Converting pgn list and headers to dataframe

In [241]:
chess_list_of_dicts = []
for i,pgn in enumerate(pgn_list):
    print(len(pgn_list))

    for j in range(len(pgn)):
        print(i,'-------',pgn[i])

        if pgn[j][0] != '':
            if len(pgn[i]) == 2:
                chess_list_of_dicts.append({pgn[j][0]:pgn[j][1]})
            elif len(pgn[i]) == 1:
                chess_list_of_dicts.append({'Chess Moves':pgn[j][0]})
        # print({pgn[i][0]:pgn[i][1]})

# df_pgn = pd.DataFrame(data=None,columns=headers)
# df_pgn

2498
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
0 ------- ['Event', 'Live Chess']
2498
1 ------- ['Site', 'Chess.com']
1 ------- ['Site', 'Chess.com']
1 ------- ['Site', 'Chess.com']
1 ------- ['Site', 'Chess.com']
1 ------- ['Site', 'Chess.com']
1 ------- ['Site', 'Chess.com']
1 ------- ['Site

IndexError: list index out of range

In [122]:
# for i in range(len(pgn_list)):
#     for j in range(len(pgn_list[i])):
#         pgn_list[i][j] =  pgn_list[i][j]
#         # print(pgn_list[i][j])
    # for i in range(len(pgn)):
    #     pgn[i]
    #     print(col)
# pgn_list[1][2]

[Event "Live Chess"]
[Site "Chess.com"]
[Date "2023.03.03"]
[Round "-"]
[White "sammasterofsleep"]
[Black "filiplivancic"]
[Result "0-1"]
[CurrentPosition "R7/8/4p3/3pP3/3PkPp1/6P1/Pn3K2/8 w - -"]
[Timezone "UTC"]
[ECO "C02"]
[ECOUrl "https://www.chess.com/openings/French-Defense-Advance-Nimzowitsch-System-4...Nc6"]
[UTCDate "2023.03.03"]
[UTCTime "17:06:15"]
[WhiteElo "987"]
[BlackElo "1012"]
[TimeControl "600"]
[Termination "filiplivancic won on time"]
[StartTime "17:06:15"]
[EndDate "2023.03.03"]
[EndTime "17:24:59"]
[Link "https://www.chess.com/game/live/71602356021"]

1. e4 {[%clk 0:10:00]} 1... e6 {[%clk 0:10:00]} 2. d4 {[%clk 0:09:57.5]} 2... d5 {[%clk 0:09:58.7]} 3. e5 {[%clk 0:09:42.2]} 3... c5 {[%clk 0:09:57.8]} 4. Nf3 {[%clk 0:09:36.7]} 4... Nc6 {[%clk 0:09:56.4]} 5. Bb5 {[%clk 0:09:31.8]} 5... Bd7 {[%clk 0:09:54.8]} 6. Bxc6 {[%clk 0:09:23.7]} 6... Bxc6 {[%clk 0:09:49.1]} 7. O-O {[%clk 0:09:23.1]} 7... Qb6 {[%clk 0:09:47.5]} 8. Be3 {[%clk 0:09:11.1]} 8... f5 {[%clk 0:09:44.8

In [58]:
pgn

['[Event "Live Chess"]',
 '[Site "Chess.com"]',
 '[Date "2023.03.13"]',
 '[Round "-"]',
 '[White "filiplivancic"]',
 '[Black "pashavass"]',
 '[Result "1-0"]',
 '[CurrentPosition "R6r/1kprp2p/Q5p1/1p6/1P6/2Pq1pPP/5P2/2K4R b - -"]',
 '[Timezone "UTC"]',
 '[ECO "A06"]',
 '[ECOUrl "https://www.chess.com/openings/Reti-Opening-Tennison-Gambit"]',
 '[UTCDate "2023.03.13"]',
 '[UTCTime "10:42:34"]',
 '[WhiteElo "788"]',
 '[BlackElo "751"]',
 '[TimeControl "180+2"]',
 '[Termination "filiplivancic won by checkmate"]',
 '[StartTime "10:42:34"]',
 '[EndDate "2023.03.13"]',
 '[EndTime "10:49:15"]',
 '[Link "https://www.chess.com/game/live/72443086645"]',
 '',
 '1. e4 {[%clk 0:03:02]} 1... d5 {[%clk 0:03:02]} 2. Nf3 {[%clk 0:03:03.6]} 2... dxe4 {[%clk 0:03:02.6]} 3. Ng5 {[%clk 0:03:04.4]} 3... f6 {[%clk 0:02:55.8]} 4. Nxe4 {[%clk 0:03:03.1]} 4... Bf5 {[%clk 0:02:55.6]} 5. Qh5+ {[%clk 0:02:59.1]} 5... Bg6 {[%clk 0:02:52.1]} 6. Qf3 {[%clk 0:02:53]} 6... Bxe4 {[%clk 0:02:44.5]} 7. Qxe4 {[%clk 0:02:52.2

## Unpacking Nested white/black dictionaries -- Creating Derived Data For Wins/Losses

In [7]:
# # Checking contents of white/black
# print('Unpacking the ratings data from all individual games')
# result_list = []
# for i in range(df.shape[0]):

#     #Find each dictionary set for white and black
#     white_result = df.iloc[i,df.columns.get_loc('white')]
#     black_result = df.iloc[i,df.columns.get_loc('black')]

#     #Search for my username, if there is a match, we will append it to the results list
#     if white_result['username'] == 'filiplivancic':
#         result_list.append(white_result)
#     else:
#         result_list.append(black_result)
    
# #Performing check to see if length of list matches the size of the dataframe
# print(df.shape[0],len(result_list))

# #Transforming results into output dataframe and dropping irrelevant columns
# result_df = pd.DataFrame(result_list)
# result_df.drop(columns={'@id','username','uuid'}, inplace=True)

# #Adding extra col to group the varieties of losses and draws into a parent catagory
# result_df['win/loss'] = ''
# for i in range(result_df.shape[0]):
#     res = result_df.iloc[i,result_df.columns.get_loc('result')]
#     if res == 'win':
#         result_df.iloc[i,result_df.columns.get_loc('win/loss')] = 'win'
#     if res == 'timevsinsufficient' or res == 'stalemate':
#         result_df.iloc[i,result_df.columns.get_loc('win/loss')] = 'draw'
#     if res != 'win' and res != 'timevsinsufficient' and res != 'stalemate':
#         result_df.iloc[i,result_df.columns.get_loc('win/loss')] = 'loss'
# # set(result_df['result'])
# # result_df.loc[result_df['win/loss'] == 'draw']
# result_df


## Concatenating Result Onto The Main Dataframe

In [8]:
# print('Appending the unpacked data onto the main dataframe')
# df_rating = pd.concat([df,result_df],axis=1)
# df_rating.head(3)

## Extracting More Metadata From The Chess Games

In [9]:
# date_list = []
# opening_list = []
# quoted = re.compile('"[^"]*"')

# for i in range(df_rating.shape[0]):
    
#     #Splitting the metadata by each new line
#     meta_data = df.iloc[i,df.columns.get_loc('pgn')].splitlines(False)
    
#     #Appending dates into the list by string index 
#     # date_list.append(meta_data[2][7:17])
#     for date in quoted.findall(meta_data[2]): date_list.append(date)

#     #Appending openings by retrieveing double quote contents
#     for opening in quoted.findall(meta_data[10]): opening_list.append(opening)


## Adding Metadata as New Columns in the DataFrame

In [10]:
# df_rating['Date'] = date_list
# df_rating['Opening'] = opening_list
# df_rating.head(5)

## Cleaning Dates and String Data

In [11]:
# # Replacing the double quotes in string
# df_rating['Date'] = df_rating['Date'].str.replace('"','')
# df_rating['Opening'] = df_rating['Opening'].str.replace('"','')

# # Converting to Datetime Format
# df_rating['Date'] = pd.to_datetime(df_rating['Date'])

# # Removing https prefix from opening column
# df_rating['Opening'] = df_rating['Opening'].str.replace('https://www.chess.com/openings/','')
# df_rating['Opening'] = df_rating['Opening'].str.replace('-',' ')
# df_rating.head(5)

## Appending the Match Date to the Result DF

In [12]:
# # Appending Date Column directly onto dataframe (index hasnt changed)
# result_df['Date'] = df_rating['Date']
# result_df.tail(10)

## Filtering Ranked 10 Minute Games

In [13]:
# #Filtering Down to Rated Games Where Time_Class = Blitz
# # df_blitz_rating = df_rating.loc[(df_rating['time_class']=='blitz') & (df_rating['time_control']=='600')]
# df_blitz_rating = df_rating.loc[ (df_rating['time_control']=='600') & df_rating['rated'] == True]
# df_blitz_rating.tail(3)

In [14]:
# #Applying to filtering to results_df and reseting index to make games appear sequentially
# result_df = result_df[result_df.index.isin(df_blitz_rating.index)].reset_index()
# result_df

## Applying Data Aggregation Techniques

In [15]:
# #Grouping Most Played Chess Openings
# df_openings = df_blitz_rating.groupby(['Opening'])['rated'].count().reset_index(name='Games')
# df_openings = df_openings.sort_values(by=['Games'],ascending=False)
# df_openings.head(10)

In [16]:
# # Grouping together Wins and Losses (For BLitz)
# def percentage_string(value):
#     return "{:.2%}".format(value)

# df_winLoss = df_blitz_rating.groupby(['win/loss'])['rated'].count().reset_index(name='Games')
# win_rate = percentage_string(df_winLoss.at[2,'Games']/df_blitz_rating.shape[0])
# loss_rate = percentage_string(df_winLoss.at[1,'Games']/df_blitz_rating.shape[0])
# draw_rate = percentage_string(df_winLoss.at[0,'Games']/df_blitz_rating.shape[0])

# #Appending Win Loss Rates onto DataFrame
# rates_list = [draw_rate,loss_rate,win_rate]
# df_winLoss['Ratio'] = rates_list

# print(f'---{player} Chess Stats--- \n{df_blitz_rating.shape[0]} Ranked Blitz Games Played:\n  Win Rate: {(win_rate)}\n  Lose Rate: {loss_rate}\n  Draw Rate: {draw_rate}\n-------------------------------')
# # df_winLoss


# Plotting the Rating Change Over Time



In [17]:
# fig, ax = plt.subplots(figsize=(10,6))
# ax.plot(result_df.index,result_df['rating'])
# ax.set_title(f'{player} Chess Rating History - Time Control: 10 Minutes',loc='left',fontsize=16)
# ax.set_xlabel('Number of Rapid Games Played', fontsize = 10)
# ax.set_ylabel('Chess Rating', fontsize=10)
# plt.tight_layout()

# #Saving Chart
# plt.savefig(pathing('pictures','rating_hist.png'),dpi=100)


## Applying a moving Average on the Rating Data

In [18]:
# #Applying 50 point moving average
# result_avg_df = result_df.copy()
# window_size = 50
# avg_data= []
# for i in range(result_avg_df.shape[0]):
#     mov_avg = result_avg_df.loc[i:(i+window_size),'rating'].mean()
#     avg_data.append(mov_avg)

# #Appending list to dataframe
# result_avg_df['average_rating'] = avg_data

# #Plotting Averaged Data
# fig, ax = plt.subplots(figsize=(10,6))
# ax.plot(result_avg_df.index,result_avg_df['average_rating'])
# ax.set_title(f'{player} Chess Rating History -  50 Point Moving Average - Time Control: 10 Minutes',loc='left',fontsize=14)
# ax.set_xlabel('Number of Rapid Games Played', fontsize = 10)
# ax.set_ylabel('Chess Rating', fontsize=10)
# plt.tight_layout()

# #Saving Chart
# plt.savefig(pathing('pictures','rating_hist_avg.png'),dpi=100)

## Initialising XlsxWriter Workbook And Exporting Tables to .xlsx

In [19]:
# #----------------------------------------------------------------------------------------------------------------------
# #--------------------------------------------.xlsx workbook initialisation---------------------------------------------
# #----------------------------------------------------------------------------------------------------------------------

# #Excel Output Filepath
# excelpath = pathing('xlsx_data','chess_data.xlsx')

# #Generating .xlsx workbook in which to record dataframes for usage.
# writer = pd.ExcelWriter(excelpath,engine='xlsxwriter')   
# workbook=writer.book
# worksheet='' #Initialise variable
# startrow = 0 #Initialise variable
# startcol = 0 #Initialise variable
# v_idx = 0    #initialise counter to index vertical chart insertions
# lr_pos = 0   #initialise left right insertion variable

# #----------------------------------------------------------------------------------------------------------------------
# #----------------------------------------------------------------------------------------------------------------------

In [20]:
# #Summary Table
# data_table = df_winLoss
# sheet_name = 'Summary'
# title_str = f'Chess Summary Stats - {player}'
# col_width_list = [20,20,20]
# [startrow,startcol,worksheet,workbook,writer] = xlsx_table_writer(data_table,sheet_name,col_width_list,title_str,startrow,startcol,worksheet,workbook,writer)

In [21]:
# #Openings Table
# data_table = df_openings
# sheet_name = 'Openings'
# title_str = f'Chess Opening Count - {player}'
# col_width_list = [40,20,20]
# [startrow,startcol,worksheet,workbook,writer] = xlsx_table_writer(data_table,sheet_name,col_width_list,title_str,startrow,startcol,worksheet,workbook,writer)

In [22]:
# #Results Table
# data_table = result_avg_df
# sheet_name = 'Rating History'
# title_str = f'Chess Rating History - {player}'
# col_width_list = [7,7,17,9,22,15]
# [startrow,startcol,worksheet,workbook,writer] = xlsx_table_writer(data_table,sheet_name,col_width_list,title_str,startrow,startcol,worksheet,workbook,writer)

In [23]:
# #Save into Excel and Exit Python out of File
# writer.close()